In [ ]:
LIB "ellipticcovers.lib";
LIB "resources.lib";
LIB "general.lib";
proc generateRing ( graph G )
{
    int n = size ( G . vertices ) ;
    int m = size ( G . edges ) ;
    ring r = 0 ,( x (1.. n ) ,q (1.. m )) , dp ;
    return ( r ) ;
}
proc generateRingV( graph G )
{
    int n = size ( G . vertices ) ;
    int m = size ( G . edges ) ;
    ring r = 0 ,( x (1.. n ) ,q (1.. m ), z(1.. n)) , dp ;
    return ( r ) ;
}
graph G = makeGraph(list(1,2,3),list(list(1,1),list(1,2),list(2,3),list(3,1),list(3,3)));
G;
ring r = generateRingV ( G) ;
r;

// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/ellipticcovers.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/parallel.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/tasks.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/resources.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/resources.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/general.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/ring.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/primdec.lib (4.2.0.0,Mar_2021)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/absfact.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/singular/LIB/triang.lib (4.1.2.0,Feb_2019)
// ** loaded /home/atraore/anaconda3/bin/../share/sing

### Sfunction

### $S (z, {aa}) = \sum_{n = 0}^{{aa}} \dfrac{2^{- 1 - n} (1 + (- 1)^n)
}{(n + 1) !} z^n = \sum_{n = 0}^{{aa}} \dfrac{{2^{- 2 n}} }{(2 n + 1) !}
z^n, {aa} \rightarrow \infty$

In [5]:
proc Sfunction(poly z, int n)
{
    int i;
    poly N;
    N=0;
        for(i=0;i<=n;i++)
        {
            N=N+(1/2^(2*i))/(fact(2*i+1))*(z^(2*i));
        }
    return(N);
}
proc  Bernoulli(int n)//from LIB "chern.lib";
{
  list  steprez;
  int i, j;
  if(n<0) // if n is negative, return the empty list
  {
    return(list());
  }
  for(i=0;i<=n;i++)
  {
    steprez=steprez+list( 1/number(i+1) );
    for(j=i;j>=1;j--)
    {
      steprez[j]=j*(steprez[j]-steprez[j+1]);
    }
  }
  return(steprez[1]);
}

## Inverse Sfunction
### $\frac{1}{S(z,aa)}=\frac{z}{2 Sinh(z/2)}= 
 \sum_{n =  0}^{aa} \left( \left\{\begin{array}{ll}
     1 & if && n = 1\\
     - \frac{- 2^n (- 2 + 2^n)}{n!} B_n & (n > = 1 && (- 1
     + n)\mod 2 = 1)
   \end{array}\right. \right) z^n $
   Where $B_n$ is Bernoulli number and $, {aa} \rightarrow \infty$.

In [6]:
proc InvSfunction(poly z, int n)
{
    int i;
    poly N;
    N=0;
        for(i=0;i<=n+1;i++)
        {
            N=N-(1/2^i)*(-2 + 2^i)*Bernoulli(i)/fact(i)*z^i;
        }
    return(N);
}


$\text{looptermV} (z_k, q_k, aa,a_k) = \sum_{w = 0}^a \sum_{w \mid a} S (w
z_k,aa)^2 w q_k^{a_k}$


In [ ]:
proc looptermV(poly z,poly q, int aa, int a)
{
    int i;
    poly p,S1,S2;
    if(a != 0)
    {
        for(i=1;i<=a;i++)
        {
            if((a mod i)==0)
            {
                S1=Sfunction(i*z,aa);
                p=p+(S1^2)*i*q^a;
            }
        }
    }
    else
        {
            p=0;
        }
    return(p);
}

## Constant Terms $a_i=0$.
### $Constterm (x_{k_1}, x_{k_2}, N) = \sum_{w = 1}^{\infty} w \left(
   \dfrac{x_{k_1}}{x_{k_2}} \right)^w = \frac{{\sum_{w = 1}^N}_{}  w (x_{k_1
   }^{N + w} x_{k_2}^{N - w})}{x_{k_1}^N x_{k_2}^N}$

In [7]:
proc constterm(def x1, def x2, int N)
{
    int w;
    poly p=0;
    for (w=1; w<=N;w++)
    {
        p=p+w*x1^(N+w)*x2^(N-w);
    }
    return (p);
}


## Non constant Terms $a_i\neq 0$
### $ {Terms} (x_{k_1}, x_{k_2}, q_k, a, N) = \sum_{w \mid d}  w \left(
   \left( \dfrac{x_{k_1}}{x_{k_2}} \right)^w + \left( \dfrac{x_{k_2}}{x_{k_1}}
   \right)^w \right) = \dfrac{\sum_{w \mid a}  w (x_{k_1 }^{N + w} x_{k_2}^{N
   - w} + x_{k_1 }^{N - w} x_{k_2}^{N + w})}{x_{k_1}^N x_{k_2}^N} q_k^a$

In [8]:
 proc term ( def x1 , def x2 , def q , int a , int N )
{
    int w;
    poly p = 0;
    for ( w = 1; w <= a ; w ++)
    {
        if ( a % w == 0)
        {
        p = p + w* ( x1 ^( N + w ) * x2 ^( N - w ) + x1 ^( N - w ) * x2 ^( N + w ) )*q ^(2* a ) ;
        }
    }
    return ( p ) ;
}

####  ${TermV} (x_{k_1}, x_{k_2}, z_{k_1}, z_{k_2}, q_k, a_k,
{aa}, N) = \sum_{w \mid a_k}  w \left( \left( \dfrac{x_{k_1}}{x_{k_2}}
\right)^w + \left( \dfrac{x_{k_2}}{x_{k_1}} \right)^w \right) = \dfrac{\sum_{w
= 0}^a \sum_{w \mid a_k}  w (x_{k_1 }^{N + w} x_{k_2}^{N - w} + x_{k_1 }^{N -
w} x_{k_2}^{N + w})}{x_{k_1}^N x_{k_2}^N} q_k^{a_k}$

In [ ]:
proc termV ( def x1 , def x2 ,  def z1 , def z2 , def q , int a ,int aa, int N )
{
    int w;
    poly p = 0;
    poly S1,S2;
    for ( w = 1; w <= a ; w ++)
    {
        if ( a % w == 0)
        {
            S1=Sfunction(w*z1,aa);
            S2=Sfunction(w*z2,aa);
            p = p + S1*S2*w* ( x1 ^( N + w ) * x2 ^( N - w ) + x1 ^( N - w ) * x2 ^( N + w ) )*q ^a ;
        }
    }
    return ( p ) ;
}

${ConsttermV} (x_{k_1}, x_{k_2}, z_{k_1}, z_{k_2}, {aa}, N) =
\sum_{w = 1}^{\infty} S (w z_{k_1}, {aa}) S (w z_{k_2}, {aa}) w
\left( \dfrac{x_{k_1}}{x_{k_2}} \right)^w = \frac{{\sum_{w = 1}^N}_{}  S (w
z_{k_1}) S (w z_{k_2}) w (x_{k_1 }^{N + w} x_{k_2}^{N - w})}{x_{k_1}^N
x_{k_2}^N}$ 

In [ ]:
proc consttermV(def x1 , def x2 ,  def z1 , def z2 ,int aa, int N)
{
    int w;
    poly p=0;
    poly S1,S2;
    for (w=1; w<=N;w++)
    {
        S1=Sfunction(w*z1,aa);
        S2=Sfunction(w*z2,aa);
        p=p+S1*S2*w*x1^(N+w)*x2^(N-w);
    }
    return (p);
}

$\text{if}$ $  a_k = 0 \Rightarrow \text{sgnV} (G, a) = \left\{\begin{array}{lll}
  - 1 & \text{if} & x_{k_1} < x_{k_2}\\
  0 & \text{if} & x_{k_1} > x_{k_2}\\
  - 2 & \text{if} & x_{k_1} = x_{k_2}
\end{array}\right.$

In [ ]:
proc sgnV(graph G, list p, list a)
{
    list b, e;
    int i;
    e = G.edges;
    for(i = 1; i <= size(a); i++)
    {
     if(a[i] == 0 && e[i][1]!= e[i][2])
     {
         if(preimg(p,e[i][1]) < preimg(p, e[i][2]))
         { 
             b[i] =-1; 
         }
         else 
         {
             b[i]=0; 
         }
     }
      if (e[i][1] == e[i][2])
     {
         b[i]=-2;
     }
      if (a[i] != 0 && e[i][1]!= e[i][2])
     { 
         b[i] = a[i];
     }
    }
    return(b);

}

 $\text{Propagator}
(x, q, z, a , \text{aa}) =\sum \left\{\begin{array}{ll}
  \prod \text{ConsttermV} (x_{k_1}, x_{k_2}, z_{k_1}, z_{k_2}, \text{aa},
  \text{sum} (a)) & \text{sgnV} (G, a_k) = - 1\\
  \prod \text{ConsttermV} (x_{k_2}, x_{k_1}, z_{k_1}, z_{k_2}, \text{aa},
  \text{sum} (a)) & \text{sgnV} (G, a_k) = 0\\
  \text{looptermV} (z_k, q_k, a_k) & \text{sgnV} (G, a_k) = - 2\\
  \text{TermV} (x_{k_1}, x_{k_2}, z_{k_1}, z_{k_2}, q_k, a_k, \text{aa},
  \text{sum} (a)) & \text{else}
\end{array}\right.$ 

$\text{SpecificfeynmanV} (x, q, z, a , \text{aa}, l, g) =
\text{Coeff}_{[x_1^{2 g [1]}, \ldots, x_{2 g - 2}^{2 g [2]}]}
\text{Coeff}_{[x_1^{0 + l [1] + N}, \ldots, x_{2 g - 2}^{0 + l [2] + N}]}
\text{Propagator} (x, q, z, a , \text{aa})$

$\text{Specificfeynmano} (x, q, z, a, o , \text{aa}, l, g) =
\text{Coeff}_{[x_1^{2 g [1]}, \ldots, x_{2 g - 2}^{2 g [2]}]}
\text{Coeff}_{[x_1^{0 + l [1] + N}, \ldots, x_{2 g - 2}^{0 + l [2] + N}]}
\text{Propagator} (x, q, z, a, o , \text{aa})$ 
$\text{Where}$ $o$ $\text{is a fixed ordering of }$ $a$ 


$\text{feynmannIntegralo} (x,
q, z, o, d , \text{aa}, l, g) = \sum_{i = 0}^d \text{Specificfeynmano} \left(
{x, q, z, o, a_i} , \text{aa}, l, g \right)$

$\text{feynmannIntegralV} (x, q, z, d , \text{aa}, l, g) = \sum_{i = 0}^d
\text{SpecificfeynmanV} \left( {x, q, z, a_i} , \text{aa}, l, g \right)$

In [9]:
proc partition (int n, int N) 
{//combination of n elements in a list of size N
    int i ,j , count ;
    list L , tmp ;
    if ( N == 1) 
    { 
        return ( list ( list ( n ) ) ) ; 
    }
    else
    {
        count = 1;
        for ( i = 0; i <= n ; i ++)
        {
            tmp = partition (n -i ,N -1) ;
            for ( j = 1; j <= size ( tmp ) ; j ++)
            {
                L [ count ] = list ( i ) + tmp [ j ];
                count = count + 1;
            }
        }
        return ( L ) ;
    }
}




proc coeftermX ( graph G , poly p ,list l, int d )
{
    int i ;
    list e , L ;
    ideal f ;
    matrix m ;
    e = G.edges ;
    for ( i = 1; i <= size ( G . vertices ) ; i ++) { L [ i ] = 0; }
    for ( i = 1; i <= size ( e ) ; i ++)
    {
        if ( e[i ][1]== e[ i ][2] )
         {
         }
        else
         {
            L [ e [ i ][1]] = L [ e [ i ][1]] + d ;
            L [ e [ i ][2]] = L [ e [ i ][2]] + d ;
         }
    }
    for ( i = 1; i <= size ( L ) ; i ++)
    {
        f = p;
        m = coeffs (p , x ( i ) ) ;
        if ( nrows ( m ) <= L [ i ]) 
        { 
            return(0) ;
        }
        p = m [l[i]+ L [ i ]+1 ,1];
    }
    return(p);
}

proc coeftermZ ( graph G , poly p ,list g)
{
    int i,nn,n,gg;
    list e , L ;
    ideal f ;
    matrix m ;
    n=2*size ( G .vertices) + size ( G.edges );
    nn=size ( G . vertices )+ size ( G.edges )+1;
    if ( p==0 )
     {
         return (p);
     }
         else
     {    
        for ( i = 1; i <= size ( g ) ; i ++)
        {
            f = p;
            m = coeffs (p , z(i) );
            gg=g[i] ;
            p = m [2*gg+1,1];
        }
    }
    return(p);
}
proc compareL(list a, list b)
{
    int i, c;
    c=1;
    i=1;
    if(size(a)!=size(b))
        {c=0;}
    while(c!=0 && i<= size(a))
    {
        if(a[i]!=b[i]){c=0;}
        i=i+1;
    }
    return(c);
 }
    proc fact ( int n )
{
    if ( n == 0) 
        { return (1) ; 
        }
    else { return ( n * fact (n -1) ) ; }
}
proc preimg(list d, def x)
{
    int i, j;
    for(i = 1; i <= size(d); i++)
     {
         if(d[i]==x)
         {
             j=i;
         }
     }
    return(j);
}   
proc sgn(graph G, list p, list a)
{
    list b, e;
    int i;
    e = G.edges;
    for(i = 1; i <= size(a); i++)
    {
         if(a[i] == 0)
        {
             if(preimg(p,e[i][1]) < preimg(p, e[i][2]))
             { 
                 b[i] =-1; 
             }
             else 
             {
                 b[i]=0; 
             }
         }
         else 
         { 
             b[i] = a[i];
         }
    }
 return(b);
}

    
proc flip(graph G, list a)
{
    list l, p, b, e, tmp; 
    int i, j, comp ,k;
    e= G.edges;
    for(i = 1; i <= size(G.vertices); i++) 
        { l[i] = 0; }
    for(i = 1; i <= size(a); i++) 
        { 
            if(a[i]==0) 
            { 
                l[e[i][1]] =1; l[e[i][2]] = 1; 
            }
        }
    for ( i = 1; i <= size ( l ) ; i ++) 
        { 
            if ( l [ i ]==1) 
                { 
                    p = p + list ( i) ; 
                }
        }
    p = permute(p);
    k=0;
    for ( i = 1; i <= size ( p ) ; i ++)
    {   
        tmp = sgn (G , p [ i ] , a ) ;
        comp = 0;
        j=1;
        while(!comp && j<=size(b)) 
        {
            if(compareL(tmp,b[j][2])) 
            {
                comp = 1;
                b[j][1] = b[j][1] + 1; 
                
            }
            else 
            { j++; 
            } 
        }
          if (! comp ) 
          { 
          b [ j ] = list (1 , tmp ) ; 
          }
    }
    for ( i = 1; i <= size ( b ) ; i ++)
    {
        b [ i ][1] = b [ i ][1] * fact ( size ( G . vertices ) ) div size ( p ) ;
    }
        return(b);
}
proc flipV(graph G, list a)
{
    list l, p, b, e, tmp; 
    int i, j, comp ,k;
    e= G.edges;
    for(i = 1; i <= size(G.vertices); i++) 
        { l[i] = 0; }
    for(i = 1; i <= size(a); i++) 
        { 
            if(a[i]==0 && e[i][1]!= e[i][2]) 
            { 
                l[e[i][1]] =1; 
                l[e[i][2]] =1; 
            }
        }
    for ( i = 1; i <= size ( l ) ; i ++) 
        { 
            if ( l [ i ]==1) 
                { 
                    p = p + list ( i) ; 
                }
        }
    p = permute(p);
    k=0;

    for ( i = 1; i <= size ( p ) ; i ++)
    {   
        tmp = sgnV (G , p [ i ] , a ) ;
        comp = 0;
        j=1;
        while(!comp && j<=size(b)) 
        {
            if(compareL(tmp,b[j][2])) 
            {
                comp = 1;
                b[j][1] = b[j][1] + 1; 
            }
            else 
            { j++; 
            } 
        }
          if (! comp ) 
          { 
          b [ j ] = list (1 , tmp ) ; 
          }
    }
    for ( i = 1; i <= size ( b ) ; i ++)
    {
        b [ i ][1] = b [ i ][1] * fact ( size ( G . vertices ) ) div size ( p ) ;
    }
        return(b);
}
    proc flipo(graph G, list a,list o)
{
    list b, tmp; 
    tmp = sgnV (G  , a,o ) ;
    b [ 1 ] = list (1 , tmp ) ; 
    return(b);
}
    proc specificFeynmanIntegral(graph G, list a)
{
 list e,f;
 poly p =0;
 poly tmp;
 int i, j;
 
 e = G.edges;
 f = flip(G, a);
 for(i = 1; i <=size(f); i++)
     {
         tmp=1;
         for(j = 1; j <= size(f[i][2]); j++)
             {
                 if (f[i][2][j] < 1)
                     {
                        if(f[i][2][j] == -1) 
                        { 
                            tmp = tmp * constterm(x(e[j][1]), x(e[j][2]), sum(a)); 

                        }
                        else 
                            { 
                                tmp = tmp * constterm(x(e[j][2]), x(e[j][1]), sum(a)); 
                            }
                     }
                 else 
                     { 
                         tmp = tmp * term(x(e[j][1]), x(e[j][2]), q(j),f[i][2][j], sum(a)); 
                     }
             }
         p=p+f[i][1]*coefterm(G,tmp,sum(a));
     }
    return (p);
 } 
proc specificFeynmanIntegralo(graph G, list a,list o,int aa,list l,list g)
{
    list e,f;
    poly p =0;
    poly tmp,pp;
    int i, j,k;
    e = G.edges;
    f = flipo(G, o,a);
    poly sz=1;
    for (k=1;k<=size(l);k++)
    {
        sz=sz*InvSfunction(z(k),aa);
    }
    for(i = 1; i <=size(f); i++)
     {
         tmp=1;
         for(j = 1; j <= size(f[i][2]); j++)
             {
                  if(f[i][2][j] == -1) 
                     {
                            tmp = tmp * consttermV(x(e[j][1]), x(e[j][2]), z(e[j][1]), z(e[j][2]),aa,sum(a)); 
                      }
                  if(f[i][2][j] == 0) 
                            { 
                                tmp = tmp * consttermV(x(e[j][2]), x(e[j][1]), z(e[j][1]), z(e[j][2]), aa,sum(a)); 
                            }
                  if(f[i][2][j] == -2)
                         {
                             tmp = tmp *looptermV(z(e[j][1]),q(j),aa,a[j]) ;
                         }
                 if(f[i][2][j] >0)            
                     { 
                         tmp = tmp * termV(x(e[j][1]), x(e[j][2]),z(e[j][1]), z(e[j][2]), q(j),f[i][2][j],aa, sum(a)); 
                     }
             }
         p=p+f[i][1]*coeftermX(G,tmp,l,sum(a));
         if(i==size(f))
         {
             pp=sz*p;
             p=coeftermZ(G,pp,g);
         }
    }
    return (p);
} 
proc specificFeynmanIntegralV(graph G, list a,int aa,list l,list g)
{
    list e,f;
    poly p =0;
    poly tmp,pp;
    int i, j,k;
    e = G.edges;
    f = flipV(G, a);
    poly sz=1;
    for (k=1;k<=size(l);k++)
    {
        sz=sz*InvSfunction(z(k),aa);
    }
     for(i = 1; i <=size(f); i++)
     {
         tmp=1;
         for(j = 1; j <= size(f[i][2]); j++)
             {
                  if(f[i][2][j] == -1) 
                     {
                            tmp = tmp * consttermV(x(e[j][1]), x(e[j][2]), z(e[j][1]), z(e[j][2]),aa,sum(a)); 
                      }
                  if(f[i][2][j] == 0) 
                            { 
                                tmp = tmp * consttermV(x(e[j][2]), x(e[j][1]), z(e[j][1]), z(e[j][2]), aa,sum(a)); 
                            }
                  if(f[i][2][j] == -2)
                         {
                             tmp = tmp *looptermV(z(e[j][1]),q(j),aa,a[j]) ;
                         }
                 if(f[i][2][j] >0)           
                     { 
                         tmp = tmp * termV(x(e[j][1]), x(e[j][2]),z(e[j][1]), z(e[j][2]), q(j),f[i][2][j],aa, sum(a)); 
                     }
             }
         p=p+f[i][1]*coeftermX(G,tmp,l,sum(a));
         if(i==size(f))
         {
             pp=sz*p;
             p=coeftermZ(G,pp,g);
         }    
    }
    return (p);
 } 
proc feynmanIntegral ( graph G , int d )
 {
    list a = partition( d,size ( G . edges ));
    list l;
    int i ;
    for ( i = 1; i <= size ( a ) ; i ++)
    {
        l[i]=  specificFeynmanIntegral(G, a[i]);
    }
    return ( sum(l) ) ;
 }
proc feynmanIntegralo( graph G, int d,list o,int aa,list l,list g)
 {
    list a = partition( d,size ( G . edges ));
    list L;
    int i ;
    for ( i = 1; i <= size ( a ) ; i ++)
    {
        L[i]=  specificFeynmanIntegralo(G, a[i],o,aa,l,g);
    }
    return ( sum(L) ) ;
 }
proc feynmanIntegralV( graph G, int d,int aa,list l,list g)
 {
    list a = partition( d,size ( G . edges ));
    list L;
    int i ;
    for ( i = 1; i <= size ( a ) ; i ++)
    {
        L[i]=  specificFeynmanIntegralV(G, a[i],aa,l,g);
    }
    return ( sum(L) ) ;
 }
proc feynmanIntegralParallel( graph G , int d )
 {
     list a = partition( size ( G . edges ) ,d) ;
     list arg , res ;
     int i , j ;
     poly sum = 0;
     int c = getcores() ;
     while ( size ( a ) > 0)
     {
         arg = list () ;
         for ( i = 1; i <= c && size ( a ) > 0; i ++)
         {
             arg = arg + list ( list (G , a [1]) ) ;
             a = delete(a ,1) ;
         }
         res = parallelWaitAll ("specificFeynmanIntegral " , arg );
         for ( i = 1; i <= size ( res ) ; i ++)
         {
             sum = sum + res [ i ];
         }
     }
     return ( sum ) ;
 }
proc feynmanIntegralSum ( graph G , int d )
{
    int i ;
    poly sum = 0;
    for ( i = 0; i <= d ; i ++)
    {
        sum = sum + feynmanIntegral (G , i ) ;
    }
    return ( sum );
}
proc feynmanIntegralSumV(graph G, int d,int aa,list l,list g )
 {
     int i ;
     poly sum = 0;
     for ( i = 1; i <= d ; i ++)
     {
         sum = sum + feynmanIntegralV(G,i,aa,l,g) ;
     }
     return ( sum );
 }
proc sub( poly p )
{
    int i ;
    matrix m;
    int v = size ( variables ( p ) ) ;
    for ( i = 2; i <= v ; i ++)
    {
        p = subst (p , q ( i ) ,q (1) ) ;
    }
    m=coeffs(p,q(1));
    return (sum(m)) ;
}

In [11]:
list g=(2,0,1);
list l=(0,0,0);
list a=(2,0,0,1,1);
list o=(1,2,3);
int aa=5  ;
poly dd=specificFeynmanIntegralV( G,  a, aa, l, g);
dd;

239/7680*q(1)^2*q(4)*q(5)


In [12]:
poly dd=specificFeynmanIntegralo( G,  a,o, aa, l, g);
dd;

239/15360*q(1)^2*q(4)*q(5)


In [13]:
poly ff=feynmanIntegralo(G,3,o,aa,l,g);
ff;

13/15360*q(1)*q(4)*q(5)


In [14]:
poly ff=feynmanIntegralV(G,3,1,l,g);
ff;

1/1152*q(1)*q(2)*q(5)+1/1152*q(1)*q(3)*q(5)+1/1152*q(1)*q(4)*q(5)


In [15]:
poly ft=feynmanIntegralSumV(G,3,1,l,g);
ft;

1/1152*q(1)*q(2)*q(5)+1/1152*q(1)*q(3)*q(5)+1/1152*q(1)*q(4)*q(5)


In [16]:
poly sf=sub(feynmanIntegralSumV(G,3,1,l,g));
sf;

1/384


In [17]:
poly fe=feynmanIntegral(G,3);
fe;

   skipping text from `;` error at token `)`
0


In [18]:
sub(feynmanIntegral(G,3));

   skipping text from `;` error at token `)`


In [19]:
sub(feynmanIntegralV(G,3,1,l,g));

1/384
